# Final Project CAP 4784

In [1]:
from pyspark.sql import SparkSession

In [2]:
import pyspark.sql.functions as F

In [3]:
from pyspark.sql.types import *

### Import growth_data.csv

In [4]:
df_data=spark.read.csv('growth_data.csv' \
                      ,header=True \
                      ,inferSchema=True)

In [5]:
# Verify data was imported.
df_data.count()

264

In [6]:
# Verify schema of df_data
# df_data.printSchema()

In [7]:
# df_data.fillna('0').show()
#df_data.na.drop().show()

### Import country_codes.csv

In [8]:
df_countries=spark.read.csv('country_codes.csv' \
                           ,header=True \
                           ,inferSchema=True)

In [9]:
# Verify country_codes.csv was imported
df_countries.count()

263

In [10]:
# Verify schema of df_countries
df_countries.printSchema()

### Join df_data and df_countries

In [11]:
df_combined=df_data.join(df_countries,on='Country_Code')

In [12]:
df_combined.printSchema()

### Using UDF to calculate the average growth per record

In [28]:
from pyspark.sql.functions import col

marksColumns = [col('1960'), col('1961'),col('1962'),col('1963'),col('1964'),col('1965'),col('1966'),col('1967') \
               ,col('1968'),col('1969'),col('1970'),col('1971'),col('1972'),col('1973'),col('1974'),col('1975') \
               ,col('1976'),col('1977'),col('1978'),col('1979'),col('1980'),col('1981'),col('1982'),col('1983') \
               ,col('1984'),col('1985'),col('1986'),col('1987'),col('1988'),col('1989'),col('1990'),col('1991') \
               ,col('1992'), col('1993'),col('1994'),col('1995'),col('1996'),col('1997') \
               ,col('1998'), col('1999'),col('2000'),col('2001'),col('2002'),col('2003'),col('2004'),col('2005') \
               ,col('2006'),col('2007'),col('2008'),col('2009'),col('2010'),col('2011'),col('2012'),col('2013') \
               ,col('2014'),col('2015'),col('2016'),col('2017'),col('2018')]

averageFunc = sum(x for x in marksColumns)/len(marksColumns)

df_average=df_combined.withColumn('Result(Avg)', averageFunc)

In [29]:
df_average.select(['Country_Code','Result(Avg)']).show(300)

+------------+--------------------+
|Country_Code|         Result(Avg)|
+------------+--------------------+
|         ABW|  1.1874106581084072|
|         AFG|   2.435532836257381|
|         AGO|  2.9576449299854315|
|         ALB|   1.030024042742419|
|         AND|  3.0818893379473686|
|         ARB|                null|
|         ARE|  7.9739998445704146|
|         ARG|  1.3429565033807718|
|         ARM|  0.8309020718417667|
|         ASM|  1.7438285869556542|
|         ATG|   1.004800500788169|
|         AUS|  1.5447155198103075|
|         AUT|  0.3940883330164894|
|         AZE|  1.6443548113937243|
|         BDI|   2.380657361464993|
|         BEL| 0.38642249242742616|
|         BEN|    2.65343112351896|
|         BFA|  2.4103854162630385|
|         BGD|  2.1014523921056996|
|         BGR|-0.17598535868735332|
|         BHR|  3.9044899934951203|
|         BHS|   2.214459234758373|
|         BIH|  0.0829386435497023|
|         BLR|  0.2637897630191854|
|         BLZ|  2.4645651921